# Participation d'organisations à des événements

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import sqlite3 as sql

In [2]:
### Define the path to the file (a string in a variable 'db_file')
# if the data directory label is at the same level as the notebook,
# then the path is 'data/astronomers_import.db'
db_file_address = '../../../Data_base_donnee/Base_donnee/cycle_manifestation.db'

In [3]:
### This ist the query that produces the data to be analysed.
# It has to be adapted to each use case
q1 = """
SELECT 
    oa.nom nom_organis,
    e.pk_evenement, 
    e.nom AS nom_evenement, 
    e.date, 
    e.heure_debut ,
    e."type" AS type_evenement,
    l.nom AS nom_lieu, 
    l.ville, 
    l.pays
FROM 
   liaison_pres_organisation_evenement lpoe 
JOIN organisation_association oa 
	ON lpoe.fk_organisation = oa.pk_organisation 
JOIN evenement e 
	ON e.pk_evenement = lpoe.fk_evenement
JOIN 
    lieu l ON l.pk_lieu = e.fk_lieu_debut
ORDER BY e.date, e."type" ;   
"""

In [4]:
### Create a connection to the SQLite database of your repository
# Beware : if the database does not exist at the specified address (db_file_address)
# it will be created and will be empty. Therefore, carefully check the database address
try:
    cn = sql.connect(db_file_address)    
except Exception as e:
    # if the connection does not work
    # an error message is raised
    print(e)    


In [5]:
### This instruction executes the query 
# then closes the connection to the database 
q1_df= pd.read_sql_query(q1, cn)
cn.close()
# Inspect the first lines
q1_df.head()

,nom_organis,pk_evenement,nom_evenement,date,heure_debut,type_evenement,nom_lieu,ville,pays
0,CFDT-Lip,34,regroupement devant le consulat suisse des ouv...,1973-05-10,None,regroupement de rue /manifestation statique,Consulat suisse,Besançon,France
1,Personnel Lip,34,regroupement devant le consulat suisse des ouv...,1973-05-10,None,regroupement de rue /manifestation statique,Consulat suisse,Besançon,France
2,CGT-Lip,34,regroupement devant le consulat suisse des ouv...,1973-05-10,None,regroupement de rue /manifestation statique,Consulat suisse,Besançon,France
3,CFDT-Lip,1,manifestation des ouvrier.ère.s de Lip,1973-05-18,11:00,manifestation de rue,Place du Port,Neuchâtel,Suisse
4,Comité d'action (CA),1,manifestation des ouvrier.ère.s de Lip,1973-05-18,11:00,manifestation de rue,Place du Port,Neuchâtel,Suisse


In [82]:
### Regrouper par date et pays, compter
g1_q1 = q1_df[['nom_organis', 'pays']].groupby(by=['nom_organis', 'pays'])\
    .size().to_frame('effectif').reset_index().sort_values(by='effectif', ascending=False)

g1_q1.head()


,nom_organis,pays,effectif
40,Ligue marxiste révolutionnaire (LMR),Suisse,13
17,Comité de travailleurs horlogerie - métallurgi...,Suisse,8
61,Personnel Lip,France,8
2,CFDT-Lip,France,7
6,CGT-Lip,France,6


In [93]:
pa = g1_q1.sort_values(by=['effectif'], ascending=True)

In [95]:
### Use a different plotting library: Plotly Express
# Observe the default behaviour of the library

### Colors
# https://stackoverflow.com/questions/63460213/how-to-define-colors-in-a-figure-using-plotly-graph-objects-and-plotly-express

fig = px.bar(pa, y='nom_organis', x='effectif', 
             color='pays',
             color_discrete_map={"Suisse": 'red', "France": 'blue'},
             # color_discrete_sequence=px.colors.qualitative.D3, 
             orientation='h')

#fig = px.bar(x=q1_df.index, y=q1_df.values, title="Manifestations", barmode='relative', orientation='h')
fig.update_yaxes(dtick=1)
fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
)
fig.show()

## Visualisation avec des points (ou bulles) par organisation et pays de l'événement 

Ces visualisations visent à mettre mieux en évidencel participation des organisation aux événements et inspecter leur chronologie

In [96]:
### Regrouper par date et pays, compter
g1_q1 = q1_df[['nom_organis', 'date', 'pays', 'nom_lieu']].groupby(['nom_organis','date', 'pays'])\
        .agg({'nom_lieu':[', '.join,'count']}).reset_index()

g1_q1.tail()


nom_organis        date    pays  \
                                                      
113               Personnel Lip  1973-06-23  Suisse   
114               Personnel Lip  1973-08-14  France   
115               Personnel Lip  1973-09-29  France   
116  Rupture pour le communisme  1973-08-23  Suisse   
117  Typographia section Zürich  1973-08-31  Suisse   

                               nom_lieu        
                                   join count  
113  Place de la Gare, Maison du Peuple     2  
114                           Usine Lip     1  
115                       Châteaufarine     1  
116                Salle des 22-Cantons     1  
117                           Volkshaus     1

In [111]:
def insert_break_after(text, n):
    if len(text) <= n:
        return text
    # If the text is longer, find the first space after the 40th character
    else:
        space_index = text.find(' ', n)
        
        print(space_index)
        if space_index == -1:
            return text
        return text[:space_index] + '<br>' + text[space_index+1:]

In [180]:
r = insert_break_after("Fédération neuchâteloise des travailleurs du commerce,  des transports et de l'alimentation (FCTA)", 7)
print(r)

10
Fédération<br>neuchâteloise des travailleurs du commerce,  des transports et de l'alimentation (FCTA)


In [191]:
def splitTextToNWords(string, n):
    words = string.split()
    out_words = ''
    i = 1
    j = 1
    for w in words:
        if i == n and j < len(words):
            i = 1
            # do not add the / for br
            out_words += w + '<br>'
        elif j < len(words):
            out_words += w + ' '
            i += 1
        else:
            out_words += w


    return out_words

In [192]:
d = "Fédération neuchâteloise des travailleurs du commerce,  des transports et de l'alimentation (FCTA)"
print(splitTextToNWords(d, 7))

Fédération neuchâteloise des travailleurs du commerce, des<br>transports et de l'alimentation (FCTA) 


In [202]:
g1_q1['nom_organis_break'] = g1_q1.nom_organis.apply(lambda x: splitTextToNWords(x, 5))
g1_q1.columns=[	'nom_organis','date','pays','noms_lieux', 'effectif','nom_organis_break']
g1_q1.tail(10)

,nom_organis,date,pays,noms_lieux,effectif,nom_organis_break
108,Parti socialiste ville de Zürich,1973-08-31,Suisse,Volkshaus,1,Parti socialiste ville de Zürich<br>
109,Personnel Lip,1973-05-10,France,Consulat suisse,1,Personnel Lip
110,Personnel Lip,1973-05-18,France,"Morteau, Villers-le-Lac, Consulat suisse",3,Personnel Lip
111,Personnel Lip,1973-05-29,France,Consulat suisse,1,Personnel Lip
112,Personnel Lip,1973-06-15,France,Place Battant,1,Personnel Lip
113,Personnel Lip,1973-06-23,Suisse,"Place de la Gare, Maison du Peuple",2,Personnel Lip
114,Personnel Lip,1973-08-14,France,Usine Lip,1,Personnel Lip
115,Personnel Lip,1973-09-29,France,Châteaufarine,1,Personnel Lip
116,Rupture pour le communisme,1973-08-23,Suisse,Salle des 22-Cantons,1,Rupture pour le communisme
117,Typographia section Zürich,1973-08-31,Suisse,Volkshaus,1,Typographia section Zürich


In [209]:
### Observer présences et absences d'organisations aux événements
# Les étiquette ont été enlevées à gauche, il faut l'affichage Plotly dynamique pour voir les valeurs en survol


#size = [s*0.5 for s in g_q1.effectif]

fig = px.scatter(g1_q1, x='date', y='nom_organis_break', size='effectif', size_max=15, 
                color='pays', 
                color_discrete_map={"Suisse": '#ff3333', "France": '#3333ff'},
                hover_data=['date', 'effectif', 'noms_lieux'])

#fig = px.bar(x=q1_df.index, y=q1_df.values, title="Manifestations", barmode='relative')
#fig.update_yaxes(dtick=1)

fig.update_layout(
    autosize=False,
    legend=dict(x=0.01),
    margin=dict(l=20, r=20, t=20, b=20),
    paper_bgcolor="LightSteelBlue",
    width=1400,
    height=2000,
    xaxis_title="Date", yaxis_title="Organisation"
    )

xtk = g1_q1.date.to_list()

fig.update_xaxes(tickangle=-60,

                 tickmode = 'array',
                 tickvals = xtk,
                 ticktext= xtk,
                
    tickfont=dict(
                       size=9,  # Set the font size here
        color="Black"
    ))


ytk = g1_q1.nom_organis_break.to_list()
print()


fig.update_yaxes(tickangle=-10,
                 #showticklabels=False,
                 tickmode = 'array',
                 tickvals = ytk,
                 ticktext= ytk, #[], # ytk, # 
    tickfont=dict(
                       size=9,  # Set the font size here
        color="DarkBlue"
    ))

fig.write_image("pictures/evenements_par_organisation_pays_et_date_with_names.jpg")

fig.show()

In [208]:
### Observer présences et absences d'organisations aux événements
# Les étiquette ont été enlevées à gauche, il faut l'affichage Plotly dynamique pour voir les valeurs en survol


#size = [s*0.5 for s in g_q1.effectif]

fig = px.scatter(g1_q1, x='date', y='nom_organis_break', size='effectif', size_max=15, 
                color='pays', 
                color_discrete_map={"Suisse": '#ff3333', "France": '#3333ff'},
                hover_data=['date', 'effectif', 'noms_lieux'])

#fig = px.bar(x=q1_df.index, y=q1_df.values, title="Manifestations", barmode='relative')
#fig.update_yaxes(dtick=1)

fig.update_layout(
    autosize=False,
    legend=dict(x=0.01),
    margin=dict(l=10, r=10, t=10, b=10),
    paper_bgcolor="LightSteelBlue",
    width=1000,
    height=1000,
    xaxis_title="Date", yaxis_title="Organisation"
    )

xtk = g1_q1.date.to_list()

fig.update_xaxes(tickangle=-50,

                 tickmode = 'array',
                 tickvals = xtk,
                 ticktext= xtk,
                
    tickfont=dict(
                       size=9,  # Set the font size here
        color="Black"
    ))


ytk = g1_q1.nom_organis_break.to_list()
print()


fig.update_yaxes(tickangle=-20,
                 showticklabels=False,
                 tickmode = 'array',
                 tickvals = ytk,
                 ticktext= ytk, #[], # ytk, # 
    tickfont=dict(
                       size=9,  # Set the font size here
        color="DarkBlue"
    ))

fig.write_image("pictures/evenements_par_organisation_pays_et_date.jpg")

fig.show()